In [4]:
from keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.datasets import cifar10
from keras.utils import to_categorical

# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
num_classes = 10

# Preprocess the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Define Residual Block
def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x

    # First convolution
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Second convolution
    x = Conv2D(filters, kernel_size=kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)

    # Adjust the shortcut connection shape if needed
    if stride > 1:
        shortcut = Conv2D(filters, kernel_size=1, strides=stride, padding='same')(shortcut)

    # Add shortcut connection
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Define ResNet Model
def build_resnet(input_shape, num_classes):
    input_tensor = Input(shape=input_shape)

    # Initial Convolution
    x = Conv2D(64, kernel_size=7, strides=2, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Residual Blocks
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, stride=2)
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=256, stride=2)
    x = residual_block(x, filters=256)

    # Global Average Pooling
    x = GlobalAveragePooling2D()(x)

    # Output layer
    output_tensor = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output_tensor)
    return model

# Build and compile the model
input_shape = x_train.shape[1:]
model = build_resnet(input_shape, num_classes)
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=74, validation_data=(x_test, y_test))

# Evaluate the model
score = model.evaluate(x_test, y_test, verbose=0)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])


Epoch 1/10
676/676 [==============================] - 25s 23ms/step - loss: 1.3457 - accuracy: 0.5158 - val_loss: 1.4676 - val_accuracy: 0.5048
Epoch 2/10
676/676 [==============================] - 14s 21ms/step - loss: 0.8983 - accuracy: 0.6819 - val_loss: 1.0948 - val_accuracy: 0.6185
Epoch 3/10
676/676 [==============================] - 14s 21ms/step - loss: 0.6835 - accuracy: 0.7601 - val_loss: 1.1625 - val_accuracy: 0.6331
Epoch 4/10
676/676 [==============================] - 14s 21ms/step - loss: 0.5360 - accuracy: 0.8125 - val_loss: 1.1032 - val_accuracy: 0.6623
Epoch 5/10
676/676 [==============================] - 14s 20ms/step - loss: 0.4094 - accuracy: 0.8553 - val_loss: 0.9107 - val_accuracy: 0.7112
Epoch 6/10
676/676 [==============================] - 14s 21ms/step - loss: 0.3064 - accuracy: 0.8941 - val_loss: 0.9674 - val_accuracy: 0.7093
Epoch 7/10
676/676 [==============================] - 14s 21ms/step - loss: 0.2108 - accuracy: 0.9258 - val_loss: 1.0054 - val_accuracy: